In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import kcgof.cgoftest as cgof
import kcgof.cdensity as cd
import kcgof.util as util
import kcgof.kernel as kernel
import scipy
from scipy.integrate import quad
import numpy as np
import torch.distributions as dists
import torch

In [ ]:
seed = 1
n = 200
dx = 1
dy = 1
alpha = 0.05

# Zheng's method

## Null

In [ ]:
# model p
p = cd.CDGaussianOLS(torch.ones(dx), c=0, variance=1.0)

p_cs = p.get_condsource()

with util.TorchSeedContext(seed=60):
    X = torch.randn(n, dx)
    Y = p_cs.cond_pair_sample(X, seed)


In [ ]:
# zheng_test = cgof.ZhengKLTest(p, alpha)
# zheng_test.perform_test(X, Y)

# Zheng's test using Monte Carlo integration
zheng_mc = cgof.ZhengKLTestMC(p, alpha, n_mc=4000, verbose=False)
zheng_mc.perform_test(X, Y)

In [ ]:
zheng_gh = cgof.ZhengKLTestGaussHerm(p, alpha)
zheng_gh.perform_test(X, Y)

## Alternative

H1 is true

In [ ]:
slope = 1.0
bias = 0.5


r = cd.CDGaussianOLS(slope*torch.ones(dx), torch.tensor(bias), variance=1.0)
r_cs = r.get_condsource()
X = torch.randn(n, dx)
Y = r_cs.cond_pair_sample(X, seed)

zheng_mc.perform_test(X, Y)

In [ ]:
zheng_gh.perform_test(X, Y)

# MMD

In [ ]:
import freqopttest.tst as tst
import freqopttest.data as fdata

seed = 13
n = 300
dx = 2
dy = 1
alpha = 0.05

## Null

In [ ]:
ols = cd.CDGaussianOLS(torch.ones(dx), c=0, variance=1.0)
with util.TorchSeedContext(seed=seed):
    X = torch.randn(n, dx)
source = ols.get_condsource()
Y = source.cond_pair_sample(X, seed)
med_x = util.meddistance(X.numpy(), 1000)
med_y = util.meddistance(Y.numpy(), 1000)
k = kernel.PTKGauss(med_x).numpy()
l = kernel.PTKGauss(med_y).numpy()

mmdtest = cgof.MMDTest(ols, k, l, alpha=alpha, seed=seed)

In [ ]:
mmdtest.perform_test(X, Y)